## **Sentiment Analysis of Twitter Dataset**

In [ ]:
!pip install transformers ktrain

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text

## EDA

In [ ]:
data = pd.read_csv('../input/tmlcds/Twitter_Data.xls')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
# Columns which contain NULL values
data.isnull().sum()

In [ ]:
data.category.unique()

In [ ]:
# Removes NULL values from Colums

data = data[~data['category'].isnull()]
data = data[~data['clean_text'].isnull()]

##Target Encoding





In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data['temp-cat'] = encoder.fit_transform(data['category'])

In [ ]:
data[['category','temp-cat']].drop_duplicates(keep='first')


In [ ]:
data.rename(columns={'temp-cat':'target'},inplace=True)

###Splitting dataset into train and test dataset

In [ ]:
 (X_train,y_train),(X_test,y_test),preprocess   = text.texts_from_df(data,text_column='clean_text', label_columns='target',maxlen=256,preprocess_mode='bert')

In [ ]:
# List object
X_train[0].shape


In [ ]:
# Creating our model
model = text.text_classifier(name='bert',train_data=(X_train,y_train), preproc=preprocess)

### Instantiating a Learner object

In [ ]:
# Getting Learning Rate
learner = ktrain.get_learner(model=model,train_data=(X_train,y_train),val_data=(X_test,y_test),batch_size=6)

In [ ]:
learning_rate = 2e-5
epochs = 1

In [ ]:
# Fitting the model
learner.fit_onecycle(lr=learning_rate,epochs=epochs)

## Validating our Model

In [ ]:
learner.validate(val_data=(X_test, y_test))

## Using our Model for prdiction

In [ ]:
#Predicting values

predictor = ktrain.get_predictor(learner.model, preprocess)

In [ ]:
data_test = ['modi is a good leader','gas prices are soaring, are the leaders sleeping ?', 'elections are being rescheduled']

In [ ]:
predictor.predict(data_test)

In [ ]:
predictor.get_classes()

In [ ]:
print(data['clean_text'][19] , ":", predictor.predict(data['clean_text'][19]))


In [ ]:
# Saving model
predictor.save('mymodel')